In [10]:
import torch
from torch import nn
from d2l import torch as d2l

In [11]:
def vgg_block(num_convs, in_channel, out_channel):
    layers=[]
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channel, out_channel,kernel_size=3,padding=1))
        layers.append(nn.ReLU())
        in_channel = out_channel
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [12]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [20]:
def vgg(conv_arch):
    vgg_blks=[]
    in_channel=1
    # vgg part
    for conv_cnt, out_channel in conv_arch:
        vgg_blks.append(vgg_block(conv_cnt, in_channel, out_channel))
        in_channel=out_channel
    # linear part
    vgg_blks.append(nn.Flatten())
    vgg_blks.extend([nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5)])
    vgg_blks.extend([nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5)])
    vgg_blks.append(nn.Linear(4096, 10))
    return nn.Sequential(*vgg_blks)

net = vgg(conv_arch)

/Users/kki/anaconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [21]:
X = torch.randn(size=(1, 1, 256, 256))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

Sequential output shape:	 torch.Size([1, 64, 128, 128])
Sequential output shape:	 torch.Size([1, 128, 64, 64])
Sequential output shape:	 torch.Size([1, 256, 32, 32])
Sequential output shape:	 torch.Size([1, 512, 16, 16])
Sequential output shape:	 torch.Size([1, 512, 8, 8])
Flatten output shape:	 torch.Size([1, 32768])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


### Not train in CPU, try to train in GPU later